In [1]:
from hops import hdfs
import datetime
from pyspark.sql import DataFrame, Row
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime, col
import sys

#num_rows=100000
num_rows=1000000
#num_rows=10000000

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
1,application_1612427817949_0003,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
fg1_schema = StructType([
  StructField("id", IntegerType(), True),
  StructField("ts", IntegerType(), True),
  StructField("f1", StringType(), True)    
])

fg1=spark.read.csv(hdfs.project_path() + "Resources/" + str(num_rows) + "-20-1-out.csv", header=True, schema=fg1_schema)
fg1=fg1.sort(col("id"),col("ts"))
fg1.show(4)

+---+----+---+
| id|  ts| f1|
+---+----+---+
|  1|1020| f1|
|  1|1040| f1|
|  1|1060| f1|
|  1|1080| f1|
+---+----+---+
only showing top 4 rows

In [3]:
fg1.count()

20000000

In [4]:
# fg2_schema = StructType([
#   StructField("id", IntegerType(), True),
#   StructField("ts", IntegerType(), True),
#   StructField("f2", StringType(), True)
# ])
# , schema=fg2_schema
fg2=spark.read.csv(hdfs.project_path() + "Resources/" + str(num_rows) + "-20-2-out.csv", header=True)
fg2=fg2.sort(col("id"),col("ts"))
fg2.show(4)

+---+----+---+
| id|  ts| f2|
+---+----+---+
|  1|1020| f2|
|  1|1040| f2|
|  1|1060| f2|
|  1|1080| f2|
+---+----+---+
only showing top 4 rows

In [5]:
fg2.count()

20000000

In [6]:
maprdd = fg2.rdd.groupBy(lambda x:x[0]).map(lambda x:(x[0],{y[1] for y in x[1]}))
result_dict = dict(maprdd.collect()) 

In [7]:
map_broadcast = sc.broadcast(result_dict)

In [8]:
def take_closest(id, ts):
    return min(map_broadcast.value[id],key=lambda x:((ts-x) if ts >= x else sys.maxsize))
columns = ["id2","join_ts","ts2"]

my_rdd = fg1.rdd.map(lambda x: (x[0],x[1],take_closest(x[0],x[1])))
filtered_rdd = my_rdd.filter(lambda x: x[1] >= x[2])
join2=spark.createDataFrame(filtered_rdd,columns).sort(col("id2"),col("join_ts"))
join2.show()

An error was encountered:
An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 17.0 failed 4 times, most recent failure: Lost task 0.3 in stage 17.0 (TID 1006, jimeuwester-worker-5.internal.cloudapp.net, executor 10): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/srv/hops/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/srv/hops/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/srv/hops/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/srv/hops/spark/python/lib/pyspark.zip/pyspark/rdd.py", line 1354, in takeUpToNumLeft
  File "/srv/hops/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(

In [9]:
joined = fg1.join(join2,(fg1.id==join2.id2) & (fg1.ts==join2.join_ts),how="inner")
drop_cols = ['id2', 'join_ts']
inter = joined.drop(*drop_cols)
inter.show(15)

An error was encountered:
name 'join2' is not defined
Traceback (most recent call last):
NameError: name 'join2' is not defined



In [10]:
inter = inter.alias('inter')
fg2 = fg2.alias('fg2')
final = inter.join(fg2,(inter.id==fg2.id) & (inter.ts2==fg2.ts),how="inner")
#.select('inter.id', 'inter.ts', 'inter.f1', 'fg2.f2')
final.show(15)

An error was encountered:
name 'inter' is not defined
Traceback (most recent call last):
NameError: name 'inter' is not defined



In [11]:
final.count()

An error was encountered:
name 'final' is not defined
Traceback (most recent call last):
NameError: name 'final' is not defined

